In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train = pd.read_parquet("/kaggle/input/clean-datas/clean_train.gzip")

In [3]:
clean_test = pd.read_parquet("/kaggle/input/clean-datas/clean_test.gzip")

In [4]:
df_test = clean_test

  # ENCODING

## OHE

In [5]:
categorical_columns = train.select_dtypes(include=['object'])
categorical_columns = categorical_columns.columns

In [6]:
categorical_columns = ['Country', 'VFN', 'Mh', 'Man', 'Tan', 'T', 'Va', 'Ve', 'Mk', 'Cn', 'Ct',
       'Cr', 'Ft', 'Fm']
selected_categorical_columns_train = [col for col in categorical_columns if train[col].nunique() < 30]
df_one_hot_train = pd.get_dummies(train[selected_categorical_columns_train],dtype=float, drop_first=True)
clean_train = pd.concat([train, df_one_hot_train], axis=1)

In [7]:
categorical_columns = ['Country', 'VFN', 'Mh', 'Man', 'Tan', 'T', 'Va', 'Ve', 'Mk', 'Cn', 'Ct',
       'Cr', 'Ft', 'Fm']
selected_categorical_columns_test = [col for col in categorical_columns if clean_test[col].nunique() < 30]
df_one_hot_test = pd.get_dummies(clean_test[selected_categorical_columns_test],dtype=float, drop_first=True)
clean_test = pd.concat([clean_test, df_one_hot_test], axis=1)

## Impact encoding

In [8]:
impact_dicts = {}

for categorical_feature in train_cat.columns:
    # Calculer la moyenne de la valeur cible pour chaque catégorie
    category_means = clean_train.groupby(categorical_feature)['Ewltp_(g/km)'].mean()

    # Calculer la différence de la moyenne de la valeur cible entre chaque catégorie et la moyenne globale
    category_impacts = category_means - category_means.mean()

    # Stocker l'impact de la catégorie dans un dictionnaire
    impact_dicts[categorical_feature] = category_impacts.to_dict()

    # Encoder la caractéristique catégorielle dans le jeu de données d'entraînement
    clean_train['encoded_' + categorical_feature] = clean_train[categorical_feature].map(impact_dicts[categorical_feature])

    # Supprimer la caractéristique catégorielle originale du jeu de données d'entraînement
    # clean_train.drop(columns=categorical_feature, inplace=True)


NameError: name 'train_cat' is not defined

In [ ]:
clean_train.drop(columns = ['Country', 'VFN', 'Mh', 'Man', 'Tan', 'T', 'Va', 'Ve', 'Mk', 'Cn', 'Ct',
       'Cr', 'Fm', "Ft"], inplace = True )
#clean_test.drop(columns = ['Country', 'VFN', 'Mh', 'Man', 'Tan', 'T', 'Va', 'Ve', 'Mk', 'Cn', 'Ct',
#       'Cr', 'Fm', "Ft"], inplace = True )


In [ ]:

# Appliquer les mêmes encodages aux données de test
for categorical_feature in train_cat.columns:
    # Si une valeur dans le jeu de données de test n'a pas été vue dans le jeu de données d'entraînement,
    # on peut décider de la remplacer par 0 ou la moyenne globale des impacts calculés sur le jeu de données d'entraînement
    default_impact = impact_dicts[categorical_feature].get("Default", 0)

    # Encoder la caractéristique catégorielle dans le jeu de données de test
    clean_test['encoded_' + categorical_feature] = clean_test[categorical_feature].apply(lambda x: impact_dicts[categorical_feature].get(x, default_impact))

    # Supprimer la caractéristique catégorielle originale du jeu de données de test
    clean_test.drop(columns=categorical_feature, inplace=True)


In [ ]:
clean_train.columns

## CountEncoder

In [ ]:
from category_encoders import CountEncoder

categorical_columns_with_values = clean_train.select_dtypes(include=['object']).columns[clean_train.select_dtypes(include=['object']).nunique() > 1].tolist()

# Initialisez l'encodeur Count
count_encoder = CountEncoder(cols=categorical_columns_with_values)

# Appliquez Count Encoding sur le jeu de données d'entraînement
clean_train_encoded = count_encoder.fit_transform(clean_train)

In [ ]:
categorical_columns_with_values = clean_test.select_dtypes(include=['object']).columns[clean_test.select_dtypes(include=['object']).nunique() > 1].tolist()

# Initialisez l'encodeur Count
count_encoder = CountEncoder(cols=categorical_columns_with_values)

# Appliquez Count Encoding sur le jeu de données d'entraînement
clean_test = count_encoder.fit_transform(clean_test)

In [ ]:
clean_train["rapport_poids_puissance"] = clean_train["m_(kg)"]/clean_train["ec_(cm3)"]
#clean_test["rapport_poids_puissance"] = clean_test["m_(kg)"]/clean_test["ec_(cm3)"]


In [ ]:
clean_train.head()

In [ ]:
categorical_columns = train.select_dtypes(include=['object'])
categorical_columns.columns

In [ ]:
clean_train=clean_train.drop(['Country', 'VFN', 'Mh', 'Man', 'Tan', 'T', 'Va', 'Ve', 'Mk', 'Cn', 'Ct',
       'Cr', 'Ft', 'Fm'], axis=1)

In [ ]:
clean_train.head()

# Selection des variables

### Correlation

In [ ]:
correlations = clean_train.corr()["Ewltp_(g/km)"]
positive_correlations = correlations[(abs(correlations) > 0.1)].index.tolist()
clean_train_filtered = clean_train[positive_correlations]

In [ ]:
correlations

In [ ]:
positive_correlations.remove("Ewltp_(g/km)")

In [ ]:
print(positive_correlations)

### Variance Threshold

In [ ]:
clean_train.columns

In [ ]:
from sklearn.feature_selection import VarianceThreshold

explicatives = clean_train[['ID', 'm_(kg)', 'Mt', 'W_(mm)', 'At1_(mm)', 'At2_(mm)', 'ec_(cm3)',
       'ep_(KW)', 'Fuel_consumption_',
       'Ft_DIESEL/ELECTRIC', 'Ft_E85', 'Ft_ELECTRIC', 'Ft_HYDROGEN', 'Ft_LPG',
       'Ft_NG', 'Ft_NG-BIOMETHANE', 'Ft_PETROL', 'Ft_PETROL/ELECTRIC',
       'Ft_UNKNOWN', 'Ct_M1G', 'Ct_N1', 'Ct_N1G', 'Ct_N2',
       'rapport_poids_puissance']]

thresholder = VarianceThreshold(threshold=0.1) 

X_selected = thresholder.fit_transform(explicatives)

feature_mask = thresholder.get_support()

features_selected = explicatives.columns[feature_mask]

print(features_selected)


# Modele

In [ ]:
clean_train.columns

In [ ]:
X = clean_train[['m_(kg)', 'Mt', 'Fuel_consumption_', 'Electric_range_(km)',
                 'Country_NO', 'Country_PL', 'Country_SE', 'Ct_M1G', 'Cr_M1G',
                 'Ft_ELECTRIC', 'Ft_PETROL', 'Ft_PETROL/ELECTRIC', 'Fm_E',
                 'Fm_H', 'Fm_M', 'Fm_P']]
y = train["Ewltp_(g/km)"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=762)

quach

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import make_scorer, mean_absolute_error

xgb_model = xgb.XGBRegressor(
    n_estimators=3000,
    max_depth=35,  
    learning_rate=0.005, 
    colsample_bytree=0.80,  
    gamma=10,  
    reg_alpha=0.8,
    reg_lambda=0.1,
    objective='reg:squarederror',  # de base reg:squarederror
    # eval_metric="mae",
    tree_method='gpu_hist', gpu_id=1, verbosity = 1)


xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

mae_best_xgb = mean_absolute_error(y_test, y_pred)
print(f"MAE du meilleur modèle XGBoost : {mae_best_xgb}")


In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import make_scorer, mean_absolute_error

# Définir les hyperparamètres à rechercher
param_grid = {
    'n_estimators': [500],
    'max_depth': [40],
    'learning_rate': [0.2],
    'gamma':[6],
    # "sampling_method": ["uniform","gradient_based"],
    "lambda": [3]
    # "booster": ["dart","gbtree"]
}

# Créer le modèle XGBRegressor
xgb_model = xgb.XGBRegressor(verbosity=0, tree_method='gpu_hist',
                             gpu_id=0, random_state=1, eval_metric="mae",
                            objective="reg:absoluteerror")

# Définir la métrique à optimiser
scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Créer l'objet GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring=scorer, cv=2)

# Effectuer la recherche sur grille (fit sur les données d'entraînement)
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres trouvés
print("Meilleurs paramètres:", grid_search.best_params_)

best_xgb_model = grid_search.best_estimator_
y_pred = best_xgb_model.predict(X_test)

mae_best_xgb = mean_absolute_error(y_test, y_pred)
print(f"MAE du meilleur modèle XGBoost : {mae_best_xgb}")


# sauvegarde des artefacts

In [ ]:
parameters_dictionnary = {"Model" : "XGBoost", 
                          "LAMBDA":3,
                          "N_ESTIMATORS" : 200, 
                          'MAX_DEPTH' : 20,
                          'LEARNING_RATE' : 0.1,
                          "MAE" : mae_best_xgb}

In [ ]:
import os
import json
from joblib import dump



dump(best_xgb_model, f"xgboost_{round(mae_best_xgb,2)}.joblib")

# submission

In [ ]:
vars_exp = ['m_(kg)', 'Mt', 'W_(mm)', 'At1_(mm)', 'At2_(mm)', 'ec_(cm3)',
       'ep_(KW)', 'Fuel_consumption_', 'Electric_range_(km)', 'Ft_PETROL',
       'encoded_VFN', 'encoded_Mh', 'encoded_Man', 'encoded_Tan', 'encoded_T',
       'encoded_Va', 'encoded_Ve', 'encoded_Mk', 'encoded_Cn', 'encoded_Ct',
       'encoded_Cr', 'encoded_Fm', 'rapport_poids_puissance']
test_rf = clean_test[vars_exp]


In [ ]:
test_rf.columns

In [ ]:
prediction = xgb_model.predict(test_rf)

submission = df_test[["ID"]].copy()  
submission["Ewltp (g/km)"] = prediction


In [ ]:
submission

In [ ]:
submission.to_csv("xgb_2_79.csv", index=False)